### Import pandas library

In [4]:
import pandas as pd

### Read recipe json (populated from ChatGPT)

In [5]:
recipes = pd.read_json('../seeds/recipes.json', orient='records')
recipes = pd.json_normalize(recipes['recipes'], max_level=0)[['name', 'author', 'meal', 'dish', 'cuisine', 'servings', 'time_required', 'ingredients', 'directions']]
recipes = recipes.reset_index().rename(columns={'index': 'recipe_id'})

### Retrieve users from the recipe list to populate User table

In [6]:
users = recipes.drop_duplicates('author')['author'].reset_index(drop=True)
users = users.reset_index().rename(columns={'index': 'user_id', 'author': 'name'})

### Retrieve ingredients from the recipe list to populate Ingredient table

In [7]:
ingredients = recipes.explode('ingredients')
ingredients = ingredients.reset_index()
ingredients = ingredients.drop(columns='name')
split = pd.DataFrame(ingredients['ingredients'].to_list(), columns = ['name', 'amount', 'metric'])
ingredients = pd.concat([ingredients, split], axis=1)[['recipe_id', 'name', 'amount', 'metric']]
ingredients = ingredients.reset_index().rename(columns={'index': 'ingredient_id'})

### Use IDs from Ingredient and User table to add references in Recipe table

In [8]:
recipes['ingredients'] = recipes['recipe_id'].apply(lambda x: ingredients.loc[ingredients['recipe_id'] == x, 'ingredient_id'].to_list())

In [9]:
recipes['user_id'] = recipes['author'].apply(lambda x: users.loc[users['name']== x, 'user_id'].iloc[0])
recipes = recipes.drop(columns = 'author')

In [11]:
recipes.directions.iloc[0]

['In a blender, combine spinach, kale, frozen banana, green apple, kiwi, grated ginger, chia seeds, coconut water, and ice.',
 'Blend until smooth and creamy.',
 'Pour into a glass and garnish with additional chia seeds if desired.',
 'Enjoy this refreshing green detox smoothie!']

### Verify tables have been set up properly

In [174]:
# recipes.head()

In [173]:
# ingredients['name'].unique()

### QA/QC to make sure the numbers are isolated

In [163]:
ingredients['amount'].unique()

array(['2', '1', '1 can', '1 head', '3', '', '1/2', '1/4', '9', '24',
       '14.5', '15', '4', '6', '1 pound', '28 ounces', '6 ounces',
       '2 teaspoons', '1 teaspoon', '1/2 teaspoon', '15 ounces', '2 cups',
       '1/2 cup', '2 tablespoons', '14 ounces', '1 tablespoon', '1/4 cup',
       '4 cups', '1/4 teaspoon', '250', '200', 'a handful', '500', '8',
       'for dipping', '1 1/2', '400', 'for topping', '300', '100', '800',
       'for frying'], dtype=object)

### Write to tables to individual CSVs 

In [175]:
recipes.to_csv('recipes.csv')

In [176]:
ingredients.to_csv('ingredients.csv')

In [177]:
users.to_csv('users.csv')